<a href="https://colab.research.google.com/github/chiyeon01/CNN_Model_Mechanism/blob/main/pytorch/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torchinfo

In [7]:
# Module import
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchinfo
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from torch.optim import SGD, Adagrad, RMSprop, Adam, AdamW

import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm

# Test Module
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision import transforms

In [13]:
class Config:
    batch_size = 8
    FineTune = False
    shuffle = False
    first_learning_rate = 1e-3
    second_learning_rate = 1e-4
    image_size = [224, 224]
    model_name = 'efficientnet_b0'
    make_summary = False,
    callbacks = []

In [14]:
# 사전학습된 모델을 만드는 함수(create pretrained model function)
# 모든 가중치는 DEFAULT로 선언(all weights = 'DEFAULT')
def create_pretrained_model(model_name, make_summary=False):
    if model_name == 'alexnet':
        model = models.alexnet(weights='DEFAULT')
    elif model_name == 'resnet50':
        model = models.resnet50(weights='DEFAULT')
    elif model_name == 'resnet101':
        model = models.resnet101(weights='DEFAULT')
    elif model_name == 'efficientnet_b0':
        model = models.efficientnet_b0(weights='DEFAULT')
        model.fc = nn.Linear(in_features=2048, out_features=10)
    elif model_name == 'efficientnet_b4':
        model = models.efficientnet_b4(weights='DEFAULT')
    elif model_name == 'efficientnet_v2_s':
        model = models.efficientnet_v2_s(weighs='DEFAULT')
    elif model_name == 'efficientnet_v2_m':
        model = models.efficientnet_v2_m(weighs='DEFAULT')
    elif model_name == 'efficientnet_v2_l':
        model = models.efficientnet_v2_l(weighs='DEFAULT')
    elif model_name == 'convnext_base':
        model = models.convnext_base(weights='DEFAULT')
    elif model_name == 'convnext_small':
        model = models.convnext_small(weights='DEFAULT')
    else:
        print("🧨🧨 [ERROR] 모델 이름이 존재하지 않습니다. 🧨🧨")
        return None

    if make_summary:
        # 모델 정보 요약(model summary)
        print(torchinfo.summary(model, input_size=[1, 3] + Config.image_size,
                  col_names=['output_size', 'num_params', 'trainable'],
                  row_settings=['depth', 'var_names'],
                  depth=3))

    return model

In [20]:
# 1번 에포크 학습 함수(1 epoch train function)
def train_epoch(model=None, dataloader=None, loss_fn=None, optimizer=None, make_summary=False):
    if dataloader == None:
        print("🧨🧨 [ERROR] 데이터로더가 존재하지 않습니다. 🧨🧨")
        return None

    device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')

    model = model.to(device)

    # 모델 학습 설정(model train setting)
    model.train()
    with tqdm(total = len(dataloader)) as progress_bar:
        for batch_idx, (images, labels) in enumerate(dataloader):
            images = images.to(device)
            labels = labels.to(device)

            preds = model(images)
            loss = loss_fn(preds, labels)

            # 옵티마이저 초기화(optimizer initialize)
            optimizer.zero_grad()

            # 오차역전파 계산(backpropagation)
            loss.backward()

            # 학습 파라미터 업데이트(parameter Update)
            optimizer.step()

            # Progress bar Update
            progress_bar.update(1)

            if batch_idx % 20 == 0:
                print(loss.item())

In [21]:
####### test #######

# transform 생성
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
])

# 데이터셋
train_data = datasets.FashionMNIST(root='data', train=True, download=True, transform=transform)
test_data = datasets.FashionMNIST(root='data', train=False, download=True, transform=transform)

train_dataloader = DataLoader(train_data, batch_size=Config.batch_size, shuffle=Config.shuffle, pin_memory=True)
test_dataloader = DataLoader(test_data, batch_size=Config.batch_size, shuffle=False, pin_memory=True)

model = create_pretrained_model(model_name=Config.model_name, make_summary=False)

loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=Config.first_learning_rate)

train_epoch(model, train_dataloader, loss_fn, optimizer, Config.make_summary)

  0%|          | 7/7500 [00:00<03:59, 31.27it/s]

9.451732635498047


  0%|          | 27/7500 [00:00<03:57, 31.45it/s]

6.494970321655273


  1%|          | 47/7500 [00:01<03:49, 32.44it/s]

...

3.7446014881134033


 13%|█▎        | 966/7500 [00:32<03:27, 31.41it/s]

1.1375360488891602


 13%|█▎        | 968/7500 [00:32<03:37, 30.06it/s]


KeyboardInterrupt: 

In [ ]:
train_epoch()